# MLP supervised classify images

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds

## load data
1. get 3 types of datasets, including train set, validation set, and test set.
2. the validation set is part of the train set.
3. each sample is a gray sacled 28*28 image with a label.
4. 

In [4]:
(ds_train, ds_test), ds_info = tfds.load('mnist',
                                         split=['train', 'test'],
                                         as_supervised=True,
                                         with_info=True,
                                        )

TypeError: Expected binary or unicode string, got WindowsGPath('C:\\Users\\LIHUILIN\\tensorflow_datasets\\downloads\\extracted\\GZIP.cvdf-datasets_mnist_train-images-idx3-ubyteRA_Kv3PMVG-iFHXoHqNwJlYF9WviEKQCTSyo8gNSNgk.gz')

In [5]:
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

print('About the train set', x_train.shape)
print('About the validation set', x_val.shape)
print('About the test set', x_test.shape)

About the train set (50000, 28, 28)
About the validation set (10000, 28, 28)
About the test set (10000, 28, 28)


- At the end, we have **50000** train samples, **10000** validation samples, **10000** test samples.
- Each sample has 28 * 28 pixels with gray scale value from 0 to 255


In [16]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [26]:
train_x_orig = x_train

In [36]:
x_train[0].reshape(-1).shape

(784,)

In [27]:
train_x_orig.shape[0]

50000

In [30]:
a[0].shape

(784,)

In [28]:
a = train_x_orig.reshape(train_x_orig.shape[0], -1)

In [22]:
x_train[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [17]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [38]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [ ]:
from tensorflow.keras.models import Sequential

In [42]:
class MLP:
    def __init__(self, sizes, epochs, lr):
        self.sizes = sizes
        self.epochs = epochs
        self.lr = lr

        # number of nodes in each layer
        input_layer=self.sizes[0]
        hidden_1=self.sizes[1]
        hidden_2=self.sizes[2]
        output_layer=self.sizes[3]

        self.params = {
            'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
            'W2':np.random.randn(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
            'W3':np.random.randn(output_layer, hidden_2) * np.sqrt(1. / output_layer)
            }
        def sigmoid(self, x, derivative=False):
            if derivative:
                return (np.exp(-x))/((np.exp(-x)+1)**2)
            return 1/(1 + np.exp(-x))

        def softmax(self, x, derivative=False):
        # Numerically stable with large exponentials
            exps = np.exp(x - x.max())
            if derivative:
                return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
            return exps / np.sum(exps, axis=0)
        def forward_pass(self, x_train):
            params = self.params

            # input layer activations becomes sample
            params['A0'] = x_train

            # input layer to hidden layer 1
            params['Z1'] = np.dot(params["W1"], params['A0'])
            params['A1'] = self.sigmoid(params['Z1'])

            # hidden layer 1 to hidden layer 2
            params['Z2'] = np.dot(params["W2"], params['A1'])
            params['A2'] = self.sigmoid(params['Z2'])

            # hidden layer 2 to output layer
            params['Z3'] = np.dot(params["W3"], params['A2'])
            params['A3'] = self.softmax(params['Z3'])

            return params['A3']

        def backward_pass(self, y_train, output):
            
            params = self.params
            change_w = {}

            # Calculate W3 update
            error = 2 * (output - y_train) / output.shape[0] * self.softmax(params['Z3'], derivative=True)
            change_w['W3'] = np.outer(error, params['A2'])

            # Calculate W2 update
            error = np.dot(params['W3'].T, error) * self.sigmoid(params['Z2'], derivative=True)
            change_w['W2'] = np.outer(error, params['A1'])

            # Calculate W1 update
            error = np.dot(params['W2'].T, error) * self.sigmoid(params['Z1'], derivative=True)
            change_w['W1'] = np.outer(error, params['A0'])

            return change_w

        def update_network_parameters(self, changes_to_w):
            '''
              Update network parameters according to update rule from
              Stochastic Gradient Descent.

              θ = θ - η * ∇J(x, y), 
                  theta θ:            a network parameter (e.g. a weight w)
                  eta η:              the learning rate
                  gradient ∇J(x, y):  the gradient of the objective function,
                                      i.e. the change for a specific theta θ
            '''

            for key, value in changes_to_w.items():
                self.params[key] -= self.lr * value

        def compute_accuracy(self, test_data, output_nodes):
            '''
              This function does a forward pass of x, then checks if the indices
              of the maximum value in the output equals the indices in the label
              y. Then it sums over each prediction and calculates the accuracy.
            '''
            predictions = []

            for x,y in zip(x_train, y_train):
                x_flatten = x.reshape(-1)
                output = self.forward_pass(x_flatten)
                output = self.forward_pass(inputs)
                pred = np.argmax(output)
                predictions.append(pred == np.argmax(targets))

            return np.mean(predictions)

        def train(self, train_list, test_list, output_nodes):
            start_time = time.time()
            for iteration in range(self.epochs):
                for x,y in zip(x_train, y_train):
                    x_flatten = x.reshape(-1)
                    output = self.forward_pass(x_flatten)
                    print(output.shape)
                    changes_to_w = self.backward_pass(y, output)
                    self.update_network_parameters(changes_to_w)
                accuracy = self.compute_accuracy(test_list, output_nodes)
                print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                    iteration+1, time.time() - start_time, accuracy * 100
                   ))

In [43]:
dnn = MLP(sizes=[784, 128, 64, 10], epochs=10, lr=0.001)
dnn.train(train_list, test_list, 10)

AttributeError: 'MLP' object has no attribute 'train'

In [1]:
# pip install tensorflow_datasets
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os
# Preset Matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)
# Set random seed for reproducible results 
tf.random.set_seed(22)

2.1.0


In [7]:
import numpy as np

In [14]:
np.linspace(-1, 2, 4)

array([-1.,  0.,  1.,  2.])

In [9]:
np.random.randn(128, 784).shape #* np.sqrt(1. / hidden_1)

(128, 784)

In [11]:
print(np.random.randn(3, 5))

[[-0.2422568  -1.40755638  0.29129313  2.3138288   0.82457833]
 [-0.37858326 -0.68592086 -1.40851092 -0.57362354 -0.20652308]
 [-0.51057224 -0.6100814  -0.38164914 -0.84629435  0.20474879]]
